In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import xgboost as xgb
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_log_error

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

In [ ]:
def date_trans(df):
    df['Policy Start Date']= pd.to_datetime(df['Policy Start Date'])
    df['Year'] = df['Policy Start Date'].dt.year
    df['Day'] = df['Policy Start Date'].dt.day
    df['Month'] = df['Policy Start Date'].dt.month
    df['DayOfWeek'] = df['Policy Start Date'].dt.dayofweek
    df.drop('Policy Start Date' , axis =1, inplace = True)
    return df

train_df = date_trans(train_df)
test_df = date_trans(test_df)

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [ ]:
# Assume train and test are already loaded as DataFrames
for col in train_df.select_dtypes(include='object').columns:
    train_df[col] = train_df[col].astype('category')
for col in test_df.select_dtypes(include='object').columns:
    test_df[col] = test_df[col].astype('category')


AADABOOST 

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
# from sklearn.metrics import mean_squared_error, mean_squared_log_error
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# # Set random seed
# np.random.seed(42)

# # Assume train_df and test_df are your DataFrames
# df = train_df

# # Features and target
# X = df.drop('Premium Amount', axis=1)
# y = df['Premium Amount']

# # Log transform the target
# y_log = np.log1p(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned_log = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Identify categorical and numerical columns
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# # Create imputers for numerical and categorical columns
# numerical_imputer = SimpleImputer(strategy='mean')  # Use mean for numerical features
# categorical_imputer = SimpleImputer(strategy='most_frequent')  # Use most frequent for categorical features
# categorical_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)  # One hot encoding for categorical features

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_imputer, numerical_cols),
#         ('cat', Pipeline([
#             ('imputer', categorical_imputer),
#             ('encoder', categorical_encoder)
#         ]), categorical_cols)
#     ]
# )

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions
# test_predictions = np.zeros(len(X))  # Test predictions for each fold

# # Test set
# X_test = test_df
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned_log)):
#     print(f"Fold {fold + 1}/{n_splits}")
    
#     # Split data
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    
#     # Apply preprocessing to both train and valid sets
#     X_train_transformed = preprocessor.fit_transform(X_train)
#     X_valid_transformed = preprocessor.transform(X_valid)
    
#     # Model
#     model = AdaBoostRegressor(
#         base_estimator=DecisionTreeRegressor(max_depth=5),
#         n_estimators=100,
#         learning_rate=0.1,
#         random_state=42
#     )
#     model.fit(X_train_transformed, y_train)
    
#     # Predictions (log transformed)
#     log_oof_preds = model.predict(X_valid_transformed)
    
#     # Revert log transformation
#     oof_predictions[valid_idx] = np.expm1(log_oof_preds)

#     fold_rmsle = np.sqrt(mean_squared_log_error(y.iloc[valid_idx], oof_predictions[valid_idx]))
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],
#         'Actual': y.iloc[valid_idx],
#         'OOF_Pred': oof_predictions[valid_idx],
#         'Fold': fold + 1
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions (transform test data)
#     X_test_transformed = preprocessor.transform(X_test)
#     log_test_preds = model.predict(X_test_transformed)
#     test_preds_per_fold[:, fold] = np.expm1(log_test_preds)

# # Combine fold results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# # Output predictions
# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())


In [ ]:
# sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('submission.csv', index=False)
# oof_results_df.to_csv('oof_aaba.csv',index = False)
# sub.head()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Set random seed
np.random.seed(42)

# Assume train_df and test_df are your DataFrames
df = train_df

# Features and target
X = df.drop('Premium Amount', axis=1)
y = df['Premium Amount']

# Log transform the target
y_log = np.log1p(y)

# Stratify target by binning into discrete intervals
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
y_binned_log = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# Identify categorical and numerical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Create imputers for numerical and categorical columns
numerical_imputer = SimpleImputer(strategy='mean')  # Use mean for numerical features
categorical_imputer = SimpleImputer(strategy='most_frequent')  # Use most frequent for categorical features
categorical_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)  # One hot encoding for categorical features

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_imputer, numerical_cols),
        ('cat', Pipeline([
            ('imputer', categorical_imputer),
            ('encoder', categorical_encoder)
        ]), categorical_cols)
    ]
)

# Stratified K-Fold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Arrays to store predictions
oof_predictions = np.zeros(len(X))  # Out-of-fold predictions
test_predictions = np.zeros(len(X))  # Test predictions for each fold

# Test set
X_test = test_df
test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold
fold_oof_results = []  # To store ID, target, and OOF predictions

# Best hyperparameters
best_params = {
    'tree_depth': 6,
    'min_samples_split': 7,
    'n_estimators': 121,
    'learning_rate': 0.005829169740875408,
    'loss': 'square'
}

# Cross-validation loop
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned_log)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # Split data
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    
    # Apply preprocessing to both train and valid sets
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_valid_transformed = preprocessor.transform(X_valid)
    
    # Model
    model = AdaBoostRegressor(
        base_estimator=DecisionTreeRegressor(max_depth=best_params['tree_depth'],
                                             min_samples_split=best_params['min_samples_split']),
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        loss=best_params['loss'],
        random_state=42
    )
    model.fit(X_train_transformed, y_train)
    
    # Predictions (log transformed)
    log_oof_preds = model.predict(X_valid_transformed)
    
    # Revert log transformation
    oof_predictions[valid_idx] = np.expm1(log_oof_preds)

    fold_rmsle = np.sqrt(mean_squared_log_error(y.iloc[valid_idx], oof_predictions[valid_idx]))
    print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
    # Store fold results
    fold_result = pd.DataFrame({
        'ID': X.index[valid_idx],
        'Actual': y.iloc[valid_idx],
        'OOF_Pred': oof_predictions[valid_idx],
        'Fold': fold + 1
    })
    fold_oof_results.append(fold_result)
    
    # Test set predictions (transform test data)
    X_test_transformed = preprocessor.transform(X_test)
    log_test_preds = model.predict(X_test_transformed)
    test_preds_per_fold[:, fold] = np.expm1(log_test_preds)

# Combine fold results
oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# Average predictions on test data
final_test_predictions = test_preds_per_fold.mean(axis=1)

# Evaluate OOF predictions
oof_mse = mean_squared_error(y, oof_predictions)
oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

print(f"OOF Mean Squared Error: {oof_mse:.4f}")
print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# Output predictions
print("Final Test Predictions:", final_test_predictions)
print(oof_results_df.head())


In [ ]:
# Hyper parameter code for Aada Boost

# import pandas as pd
# import numpy as np
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# import optuna

# # Assuming train_df is already loaded
# # Replace this with your actual data loading code
# # train_df = pd.read_csv("your_data.csv")

# # Sample DataFrame (replace with actual dataset)
# data = {
#     "feature1": [1, 2, 3, np.nan, 5],
#     "feature2": ["A", "B", "A", "B", np.nan],
#     "target": [10, 15, 10, 25, 30]
# }
# train_df = pd.DataFrame(data)

# # Define features
# categorical_features = [col for col in train_df.columns if train_df[col].dtype == "object"]
# numerical_features = [col for col in train_df.columns if train_df[col].dtype in ["int64", "float64"] and col != "target"]

# # Preprocessor
# numerical_transformer = Pipeline(steps=[
#     ("imputer", SimpleImputer(strategy="mean")),
#     ("scaler", StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ("imputer", SimpleImputer(strategy="most_frequent")),
#     ("onehot", OneHotEncoder(handle_unknown="ignore"))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", numerical_transformer, numerical_features),
#         ("cat", categorical_transformer, categorical_features)
#     ]
# )

# # Objective function
# def objective(trial):
#     X = train_df.drop(columns=["target"])
#     y = train_df["target"]

#     # Log-transform target to stabilize variance
#     y_log = np.log1p(y)

#     # Train-test split
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y_log, test_size=0.2, random_state=42)

#     # Suggest hyperparameters
#     base_estimator = DecisionTreeRegressor(
#         max_depth=trial.suggest_int("tree_depth", 1, 10),
#         min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
#     )

#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 500),
#         "learning_rate": trial.suggest_float("learning_rate", 0.001, 1.0, log=True),
#         "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
#         "base_estimator": base_estimator
#     }

#     # Model pipeline
#     model_pipeline = Pipeline(steps=[
#         ("preprocessor", preprocessor),
#         ("regressor", AdaBoostRegressor(**params))
#     ])

#     # Fit the model
#     try:
#         model_pipeline.fit(X_train, y_train)
#         preds = model_pipeline.predict(X_valid)
#         mse = mean_squared_error(y_valid, preds)
#         return mse
#     except Exception as e:
#         print(f"Error during model fitting: {e}")
#         raise

# # Optimize
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50)

# print("Best parameters:", study.best_params)
# print("Best score:", study.best_value)

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_log_error
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.impute import SimpleImputer
# import optuna

# # Log transform function
# def log_transform(y):
#     return np.log1p(y)  # log(1 + y) for zero-safe logarithm

# # Inverse log transform function
# def inverse_log_transform(y_log):
#     return np.expm1(y_log)  # exp(y_log) - 1 for reverse transformation

# # Check if 'Premium Amount' column is present in train_df
# if 'Premium Amount' not in train_df.columns:
#     raise KeyError("'Premium Amount' column is missing from the DataFrame")

# # Separate categorical and numeric columns
# categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns
# numeric_cols = train_df.select_dtypes(include=['number']).columns

# # Impute missing values separately
# num_imputer = SimpleImputer(strategy='mean')  # Imputer for numeric data
# cat_imputer = SimpleImputer(strategy='most_frequent')  # Imputer for categorical data

# # Handle features (X)
# X = train_df.drop('Premium Amount', axis=1)
# X_numeric = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols)
# X_categorical = pd.DataFrame(cat_imputer.fit_transform(X[categorical_cols]), columns=categorical_cols)
# X = pd.concat([X_numeric, X_categorical], axis=1)

# # Handle target (y)
# y = train_df['Premium Amount']
# y = num_imputer.fit_transform(y.values.reshape(-1, 1)).ravel()  # Impute missing values in the target

# # Log transform the target variable
# y_log = log_transform(y)

# def objective(trial):
#     # Hyperparameter space to search
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step=50),
#         "learning_rate": trial.suggest_float("learning_rate", 0.001, 1.0, log=True),
#         "base_estimator": DecisionTreeRegressor(
#             max_depth=trial.suggest_int("max_depth", 1, 10),
#             min_samples_split=trial.suggest_int("min_samples_split", 2, 20),
#             min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
#         )
#     }
    
#     # Split data into train and validation sets
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y_log, test_size=0.2, random_state=42)
    
#     # Initialize the model with the given parameters
#     model = AdaBoostRegressor(**params)
    
#     # Fit the model on log-transformed target
#     model.fit(X_train, y_train)
    
#     # Predict on the validation set and inverse transform
#     y_pred_log = model.predict(X_valid)
#     y_pred = inverse_log_transform(y_pred_log)
#     y_valid_original = inverse_log_transform(y_valid)
    
#     # Calculate RMSLE on original scale
#     score = np.sqrt(mean_squared_log_error(y_valid_original, y_pred))
    
#     return score

# # Optuna Study for Hyperparameter Tuning
# study = optuna.create_study(direction="minimize")  # We want to minimize RMSLE
# study.optimize(objective, n_trials=50)

# # Print the best parameters and score
# print("Best RMSLE:", study.best_value)
# print("Best Hyperparameters:", study.best_params)


In [ ]:
# print(train_df.columns)


In [ ]:

# model_params = {'n_estimators': 7000,
#                 'learning_rate': 0.0037894924041441663,
#                 'max_depth': 9,
#                 'min_child_weight': 7,
#                 'gamma': 0.003166869962093635,
#                 'subsample': 0.8491559357878403,
#                 'colsample_bytree': 0.9931791435553496,
#                 'reg_alpha': 8.030670352805062e-08,
#                 'reg_lambda': 0.23939002451629704,
#                 "random_state": 42,
#                 'eval_metric': 'rmsle',
#                 "objective": "reg:squarederror",  # XGBoost objective for regression
#                 "tree_method": "hist",  # Use GPU acceleration
#                 "device": "cuda",
#                 "enable_categorical": True  # Enable categorical handling if required
#                }



In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_log_error
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# import optuna

# # Log transform function
# def log_transform(y):
#     return np.log1p(y)  # log(1+y) to handle zero values

# # Inverse log transform function
# def inverse_log_transform(y_log):
#     return np.expm1(y_log)  # exp(y) - 1 to revert log1p

# # Prepare data
# X = train_df.drop('Premium Amount', axis=1)
# y = train_df['Premium Amount']

# # Log transform the target variable
# y_log = log_transform(y)

# # Identify categorical and numerical features
# categorical_features = [col for col in X.columns if X[col].dtype == 'object']
# numerical_features = [col for col in X.columns if col not in categorical_features]

# # Imputation transformers for numerical and categorical features
# numerical_transformer = SimpleImputer(strategy='mean')  # Mean imputation for numerical features
# categorical_transformer = SimpleImputer(strategy='most_frequent')  # Most frequent imputation for categorical features

# # Create a column transformer to apply different imputation strategies
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_features),  # Apply mean imputation to numerical columns
#         ('cat', categorical_transformer, categorical_features)  # Apply most frequent imputation to categorical columns
#     ])

# def objective(trial):
#     # Hyperparameter space to search for AdaBoostRegressor
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 500),  # Number of boosting iterations
#         "learning_rate": trial.suggest_float("learning_rate", 0.001, 1.0, log=True),  # Learning rate for boosting
#         "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),  # Loss function for AdaBoost
#         "base_estimator": DecisionTreeRegressor(
#             max_depth=trial.suggest_int("tree_depth", 1, 10),  # Depth of the base decision tree estimator
#             min_samples_split=trial.suggest_int("min_samples_split", 2, 10)  # Minimum samples required to split a node
#         ),
#     }

#     # Create a pipeline with preprocessing (imputation) and AdaBoostRegressor
#     model_pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),  # Apply the preprocessor to impute missing values
#         ('regressor', AdaBoostRegressor(**params))  # Apply the AdaBoost model
#     ])
    
#     # Split data into train and validation sets
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y_log, test_size=0.2, random_state=42)

#     # Fit the model on log-transformed target
#     model_pipeline.fit(X_train, y_train)

#     # Predict on the validation set and inverse transform
#     y_pred_log = model_pipeline.predict(X_valid)
#     y_pred = inverse_log_transform(y_pred_log)
#     y_valid_original = inverse_log_transform(y_valid)

#     # Calculate RMSLE on original scale
#     score = np.sqrt(mean_squared_log_error(y_valid_original, y_pred))

#     return score

# # Optuna Study for Hyperparameter Tuning
# study = optuna.create_study(direction="minimize")  # We want to minimize RMSLE
# study.optimize(objective, n_trials=50)

# # Print the best parameters and score
# print("Best RMSLE:", study.best_value)
# print("Best Hyperparameters:", study.best_params)
